# Import Data

In [ ]:
import numpy as np
import tensorflow as tf
import os
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                               
from tqdm import tqdm

IMAGE_SIZE = (150, 150)
# Chargement des données d'entraînement
def load_data():
    datasets = ['C:\\Users\\etern\\Documents\\PROJET_CNN_JB_1\\seg_train\\seg_train', 'C:\\Users\\etern\\Documents\\PROJET_CNN_JB_1\\seg_test\\seg_test']

    output = []
    #
    class_names = ['mountain', 'street', 'glacier', 'buildings', 'sea', 'forest']
    class_names_label = {class_name: i for i, class_name in enumerate(class_names)}
    nb_classes = len(class_names)

#TRANSFORMATION image matric + label en vector one-hot
    for dataset in datasets:
        images = []
        labels = []
        print("Loading {}".format(dataset))

        for folder in os.listdir(dataset):
            label = class_names_label[folder]

            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                img_path = os.path.join(os.path.join(dataset, folder), file)
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE)
                images.append(image/255)
                labels.append(label)

        images = np.array(images, dtype='float32')
        labels = np.array(labels, dtype='int32')
        labels = tf.keras.utils.to_categorical(labels, nb_classes)  # Convert labels to one-hot vectors

        output.append((images, labels))

    return output

In [ ]:
(train_images, train_labels), (test_images, test_labels) = load_data()

# Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential

input_shape = (150, 150, 3)
model = keras.Sequential(
    [
        
        layers.Conv2D(32, 3, input_shape = input_shape, activation='relu', padding="same", strides=1),
        layers.Conv2D(32, 3, activation='relu', padding="same", strides=1),
        layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'),
        
        layers.Conv2D(64, 3, activation='relu', padding="same", strides=1),
        layers.Conv2D(64, 3, activation='relu', padding="same", strides=1),
        layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'),
        
        layers.Conv2D(128, 3, activation='relu', padding="same", strides=1),
        layers.Conv2D(128, 3, activation='relu', padding="same", strides=1),
        layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'),
        
        layers.Conv2D(64, 3, activation='relu', padding="same", strides=1),
        layers.Conv2D(64, 3, activation='relu', padding="same", strides=1),
        layers.MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'),
        
        layers.Reshape([-1]),
        layers.Dense(16, activation="relu"),
        layers.Dense(6, activation="softmax"),
    ]
)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics='accuracy',
                       )

model.summary()

In [ ]:
hist = model.fit(x=train_images, y=train_labels, epochs=4, batch_size=16, validation_data=(test_images, test_labels))

# Plotting Results

In [ ]:
accuracy = hist.history['accuracy']
val_accuracy = hist.history['val_accuracy']
plt.plot(accuracy, c='b', label="accuracy")
plt.plot(val_accuracy, c='r', label="validation accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
loss = hist.history['loss']
val_loss = hist.history['val_loss']
plt.plot(loss, c='b', label="loss")
plt.plot(val_loss, c='r', label="validation loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Confusion Matrix

In [ ]:
import seaborn as sn
from sklearn.metrics import confusion_matrix
y_pred = np.argmax(model.predict(test_images), axis=1)
y_true = np.argmax(test_labels, axis=1)

cm = confusion_matrix(y_true, y_pred)

classes = ['mountain', 'street', 'glacier', 'buildings', 'sea', 'forest']
plt.figure(figsize=(8, 6))
sn.heatmap(cm, annot=True, fmt="d", xticklabels=classes, yticklabels=classes)
plt.xlabel("Prédictions")
plt.ylabel("Valeurs réelles")
plt.title("Matrice de confusion")
plt.show()